# Implementing an Physics-informed neural network for the 1D Heat equation using the PINN framework

In [ ]:
import sys
import numpy as np
import scipy.io
import torch
from pyDOE import lhs
from torch import Tensor, ones, stack, load
from torch.autograd import grad
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
from torch.nn import Module

sys.path.append('.../...')  # PINNFramework etc.
import PINNFramework as pf

In [ ]:
class InitialConditionDataset(Dataset):

    def __init__(self, n0):
        """
        Constructor of the boundary condition dataset

        Args:
          n0 (int)
        """
        super(type(self)).__init__()
        data=pd.read_csv('2021-02-20_battHeater_outside_temp9.0C_IC4.66.csv')
        u = pd.DataFrame(data, columns= ['Temperature(℃)']).to_numpy()
        t = np.arange(0,u.size*60,60).reshape((u.size, 1))
        x= np.linspace(0, 1, num=40)
        x=x.reshape((x.size,1))


        idx_x = np.random.choice(x.shape[0], n0, replace=False)
        self.x = x[idx_x,:]
        self.u = u[idx_x,:]
        self.t = np.zeros(self.x.shape)

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

    def __getitem__(self, idx):
        
        x = np.concatenate([self.x, self.t], axis=1)
        y = np.concatenate([self.u], axis=1)

        return Tensor(x).float(), Tensor(y).float()

In [ ]:
class PDEDataset(Dataset):
    def __init__(self, nf, lb, ub):
        self.xf = lb + (ub - lb) * lhs(2, nf)

    def __getitem__(self, idx):
        """
        Returns data for initial state
        """
        return Tensor(self.xf).float()

    def __len__(self):
        """
        There exists no batch processing. So the size is 1
        """
        return 1

In [ ]:
def derivatives(x, u):

    grads = ones(u.shape, device=u.device) # move to the same device as prediction
    grad_u = grad(u, x, create_graph=True, grad_outputs=grads)[0]

    # calculate first order derivatives
    u_x = grad_u[:, 0]
    u_t = grad_u[:, 1]

    # calculate second order derivatives
    grad_u_x = grad(u_x, x, create_graph=True, grad_outputs=grads)[0]

    u_xx = grad_u_x[:, 0]

    x, t = x.T
    x = x.reshape(u.shape)
    t = t.reshape(u.shape)

    return torch.stack([x, t, u,
                    u_xx, u_t], 1).squeeze()

In [ ]:
class HPM(Module):

    def __init__(self, kappa_net: Module):
        """
        Constructor of the HPM Model 
        """
        super(HPM, self).__init__()
        self.kappa_net = kappa_net
  
    def forward(self, derivatives):

        kappa_net_input = derivatives[:, :2]
        kappa_output = self.alpha_net(kappa_net_input)
        kappa_output = kappa_output.view(-1)

        u_xx = derivatives[:, 4].view(-1)

        predicted_u_t = kappa_output * (u_xx)

        return predicted_u_t


In [ ]:
if __name__ == "__main__":
    # Domain bounds
    lb = np.array([0.0, 0.0])
    ub = np.array([1, 12600])


    # initial condition
    ic_dataset = InitialConditionDataset(n0=20)
    initial_condition = pf.InitialCondition(ic_dataset)

    pde_dataset = PDEDataset(20000, lb, ub)

    # Diffusivity model
    # Input: x,t
    # Output: diffusivity k
    kappa_net = pf.models.MLP(input_size=2,
                              output_size=1,
                              hidden_size=100,
                              num_hidden=4,
                              lb = lb ,
                              ub = ub)
    
    # PINN model
    # Input: x,t
    # Output: temperature u at the point
    model = pf.models.MLP(input_size=2,
                          output_size=1, 
                          hidden_size=100, 
                          num_hidden=4, 
                          lb=lb, 
                          ub=ub)

    hpm_model = HPM(kappa_net)
    hpm_loss = pf.HPMLoss.HPMLoss(pde_dataset, derivatives, hpm_model)
    pinn = pf.PINN(model, 2, 1, hpm_loss, initial_condition, boundary_condition=None, use_gpu=False)
    pinn.fit(100, 'Adam', 1e-3)
